In [4]:
from sqlalchemy import create_engine
from tqdm import tqdm
import pandas as pd
import numpy as np
import pymysql
import pickle
import csv
import os
import time

In [12]:
# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def sqlalchemy_connect_ip(ip_address, db_name):
    engine = create_engine("mysql+pymysql://admin:"
                +"big15" # user password
                +"@{0}:3306/{1}?charset=utf8".format(ip_address, db_name)
                , encoding='utf8')
    
    return engine.connect()


def get_pymysql_connection(ip_address, db_name):

    conn = pymysql.connect(host=ip_address, user='admin', password='big15'
                        , db=db_name, charset='utf8')

    return conn

def get_pymysql_stock_list(conn, db_name):

    # 현재 DB 내 존재하는 테이블(종목) 추출
    sql = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{0}'".format(db_name)

    with conn:
        with conn.cursor() as cur:
            cur.execute(sql)
            result = [item[0] for item in cur.fetchall()]
            cur.close()

            return result
               

def get_empty_day_df(sqlalchemy_conn):
     
    sql = "SELECT * FROM investing_data.aedkrw내역 where 날짜 = 20221122"
            
    result = sqlalchemy_conn.execute(sql)
    empty_day_df = pd.DataFrame(result.fetchall())
    
    empty_day_df = empty_day_df.set_index('날짜')
    empty_day_df = empty_day_df.notnull().replace(True, np.NaN)

    empty_day_df = empty_day_df.reset_index()
    empty_day_df = empty_day_df.drop(columns=['AEDKRW내역_종가','AEDKRW내역_오픈','AEDKRW내역_고가'
                                      ,'AEDKRW내역_저가','AEDKRW내역_거래량','AEDKRW내역_변동'])
    empty_day_df.to_pickle('./pickle/empty_day_df.pkl')
    
    return empty_day_df

def get_sqlalchemy_investing_data(day_df, investing_table_list):
    
    investing_df = pd.DataFrame(day_df)
    investing_df['날짜'] = investing_df['날짜'].astype(str).astype(int) 
    for table in investing_table_list:
        conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
        
        sql = "SELECT * FROM investing_data.`{0}` where 날짜 = 20221122".format(table)
        result = conn.execute(sql)
        table_df = pd.DataFrame(result.fetchall())
        table_df['날짜'] = table_df['날짜'].astype(str).astype(int)
        investing_df = pd.merge(investing_df, table_df,on='날짜', how='left')
        
    for c in list(investing_df.columns):
        if c.split('_')[-1] == '거래량' or c.split('_')[-1] == '변동':
            investing_df[c] = investing_df[c].fillna(0)
        else:
            investing_df[c] = investing_df[c].fillna(method='bfill')
            
    return investing_df  
        
def get_sqlalchemy_stock_data(conn, stock_table_list, investing_df):
    
    stock_df = pd.DataFrame()
    investing_df['날짜'] = investing_df['날짜'].astype(str).astype(int) 
    # for table in tqdm(stock_table_list):
    sql = "SELECT * FROM stock_info.`{0}` where 날짜 = 20221122".format(code)
    
    result = conn.execute(sql)
    table_df = pd.DataFrame(result.fetchall())
    # if table_df.isnull().sum() == 0:
    table_df['날짜'] = table_df['날짜'].astype(str).astype(int)
    merge_df = pd.merge(table_df, investing_df, on='날짜')
    # merge_df = merge_df.fillna(method='bfill')

    stock_df = pd.concat([stock_df, merge_df], axis=0)
    
    
    conn.close()
    return stock_df     


In [9]:
pymysql_conn = get_pymysql_connection('192.168.50.123', 'investing_data')
sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
investing_table_list = get_pymysql_stock_list(pymysql_conn, 'investing_data')
empty_day_df = get_empty_day_df(sqlalchemy_conn)

In [10]:
investing_df = get_sqlalchemy_investing_data(empty_day_df, investing_table_list)

In [26]:
# investing_df.to_csv('../cor/investing_df_update.csv', encoding='utf-8-sig')

In [28]:
# investing_df = pd.read_csv('../cor/investing_df_update.csv', encoding='utf-8-sig', index_col=0)

In [24]:
pymysql_conn = get_pymysql_connection('192.168.50.123', 'investing_data')
sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
stock_table_list = ['005930','373220','207940','000660','051910','247540','091990','066970','293490','028300']
for code in stock_table_list:
    pymysql_conn = get_pymysql_connection('192.168.50.123', 'investing_data')
    sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
    complete_df = get_sqlalchemy_stock_data(sqlalchemy_conn, code, investing_df)
    complete_df.to_pickle(f'../colume_pickle/6개월_choice_10개_대형주_update_{code}.pkl')
    
# ['003070','006220','014580','118990','027410','000060','008560','003520','067290','064350']
# get_pymysql_stock_list(pymysql_conn, 'stock_info')
    

In [29]:
complete_df_005930.loc[0,'종가']

60900

In [16]:
complete_df

,날짜,시간,시가,고가,저가,종가,거래량,거래대금,누적체결매수수량,누적체결매도수량,...,헝가리BUX내역_고가,헝가리BUX내역_저가,헝가리BUX내역_거래량,헝가리BUX내역_변동,호주SPASX내역_종가,호주SPASX내역_오픈,호주SPASX내역_고가,호주SPASX내역_저가,호주SPASX내역_거래량,호주SPASX내역_변동
0,20221122,901,60900.0,61000.0,60800.0,60900.0,146274.0,8.907390e+09,132969.0,13105.0,...,45464.78,44601.50,0,1.66,7181.3,7139.3,7194.1,7139.3,677040000,0.59
1,20221122,902,60900.0,60900.0,60700.0,60800.0,253103.0,1.538950e+10,218651.0,180526.0,...,45464.78,44601.50,0,1.66,7181.3,7139.3,7194.1,7139.3,677040000,0.59
2,20221122,903,60800.0,61000.0,60800.0,60900.0,92486.0,5.631180e+09,289757.0,201906.0,...,45464.78,44601.50,0,1.66,7181.3,7139.3,7194.1,7139.3,677040000,0.59
3,20221122,904,61000.0,61000.0,60900.0,61000.0,15629.0,9.522300e+08,294006.0,213286.0,...,45464.78,44601.50,0,1.66,7181.3,7139.3,7194.1,7139.3,677040000,0.59
4,20221122,905,61000.0,61000.0,60800.0,61000.0,232338.0,1.414887e+10,397409.0,342221.0,...,45464.78,44601.50,0,1.66,7181.3,7139.3,7194.1,7139.3,677040000,0.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900,20221128,1517,33250.0,33300.0,33250.0,33250.0,3723.0,1.238740e+08,325808.0,572409.0,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42
1901,20221128,1518,33300.0,33350.0,33250.0,33350.0,5570.0,1.854990e+08,329783.0,574004.0,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42
1902,20221128,1519,33350.0,33350.0,33250.0,33300.0,6337.0,2.110630e+08,332125.0,577999.0,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42
1903,20221128,1520,33250.0,33300.0,33250.0,33250.0,7810.0,2.598110e+08,335340.0,582594.0,...,45952.22,45484.64,0,-0.28,7229.1,7259.5,7259.5,7216.0,700990000,-0.42


In [10]:
complete_df.to_pickle('../colume_pickle/test_complete_df_6개월_choice_10개_대형주_update.pkl')

In [17]:
with open('../colume_pickle/test_complete_df_6개월_choice_10개_대형주.pkl', 'rb') as f:
    stock_df = pickle.load(f)

In [34]:
stock_df

,날짜,시간,시가,고가,저가,종가,거래량,거래대금,누적체결매수수량,누적체결매도수량,...,헝가리BUX내역_고가,헝가리BUX내역_저가,헝가리BUX내역_거래량,헝가리BUX내역_변동,호주SPASX내역_종가,호주SPASX내역_오픈,호주SPASX내역_고가,호주SPASX내역_저가,호주SPASX내역_거래량,호주SPASX내역_변동
0,20220502,901,66600.0,66800.0,66500.0,66700.0,824383.0,5.490616e+10,709457.0,114926.0,...,43468.19,42571.10,0,-1.76,7347.0,7435.0,7435.0,7301.6,565050000.0,-1.18
1,20220502,902,66700.0,66900.0,66700.0,66800.0,203314.0,1.356986e+10,789226.0,238471.0,...,43468.19,42571.10,0,-1.76,7347.0,7435.0,7435.0,7301.6,565050000.0,-1.18
2,20220502,903,66800.0,66900.0,66700.0,66800.0,190201.0,1.270068e+10,832905.0,384993.0,...,43468.19,42571.10,0,-1.76,7347.0,7435.0,7435.0,7301.6,565050000.0,-1.18
3,20220502,904,66700.0,66800.0,66600.0,66700.0,126366.0,8.426040e+09,856269.0,487995.0,...,43468.19,42571.10,0,-1.76,7347.0,7435.0,7435.0,7301.6,565050000.0,-1.18
4,20220502,905,66700.0,66700.0,66600.0,66700.0,55899.0,3.725630e+09,883780.0,516383.0,...,43468.19,42571.10,0,-1.76,7347.0,7435.0,7435.0,7301.6,565050000.0,-1.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52573,20221121,1517,35400.0,35450.0,35400.0,35450.0,2496.0,8.836100e+07,190577.0,253355.0,...,44778.83,44034.23,0,0.22,7139.3,7151.8,7174.4,7131.7,598630000.0,-0.17
52574,20221121,1518,35450.0,35450.0,35400.0,35450.0,1130.0,4.003700e+07,191269.0,253793.0,...,44778.83,44034.23,0,0.22,7139.3,7151.8,7174.4,7131.7,598630000.0,-0.17
52575,20221121,1519,35400.0,35450.0,35400.0,35450.0,2607.0,9.236600e+07,192828.0,254841.0,...,44778.83,44034.23,0,0.22,7139.3,7151.8,7174.4,7131.7,598630000.0,-0.17
52576,20221121,1520,35400.0,35450.0,35400.0,35450.0,2194.0,7.774400e+07,194375.0,255488.0,...,44778.83,44034.23,0,0.22,7139.3,7151.8,7174.4,7131.7,598630000.0,-0.17


In [35]:
# 상관 계수 높은 거 추출

corr_matrix = stock_df.corr()
cor = corr_matrix["pct_label"]
cor1 = cor[(cor.values>0.075) | (cor.values<-0.075) |
           (cor.index == '고가') | (cor.index == '시가') | 
           (cor.index == '종가') | (cor.index == '저가')  ]

cor1 = cor1.drop('pct_label')
len(cor1)

C:\Users\USER\AppData\Local\Temp\ipykernel_17176\1135314872.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = stock_df.corr()


In [20]:
cor1.to_pickle('../colume_pickle/6개월_choice_10개_cor_0.075_lstm_대형주.pkl')

In [16]:
cor1.isnull().sum()

0

In [17]:
len(cor1)

187

In [18]:
# cor2=cor.sort_values(ascending=False)
# cor2.to_csv('../colume_pickle/6개월_choice_10개_cor_0.07_lstm.csv', encoding= 'utf-8-sig')

In [19]:
col_list = list(cor1.index)
X_stock_df = stock_df.drop(['pct_label'],axis=1)
X_stock_df = X_stock_df.set_index(['날짜'])
X_stock_df = X_stock_df[col_list]
X_stock_df

,시간,시가,고가,저가,종가,거래량,전일대비,상장주식수,시가총액,외국인주문한도수량,...,프랑스6년채권수익율내역_고가,프랑스6년채권수익율내역_저가,프랑스7년채권수익율내역_종가,프랑스7년채권수익율내역_오픈,프랑스7년채권수익율내역_고가,프랑스7년채권수익율내역_저가,프랑스8년채권수익율내역_종가,프랑스8년채권수익율내역_고가,프랑스8년채권수익율내역_저가,프랑스9년채권수익율내역_종가
날짜,,,,,,,,,,,,,,,,,,,,,
20220502,901,21100.0,21100.0,20900.0,21000.0,807.0,-200,25210000,528149000000,25210438,...,1.051,0.975,1.0842,1.065,1.1020,1.032,1.217,1.241,1.154,1.341
20220502,902,20900.0,20950.0,20900.0,20950.0,10.0,-200,25210000,528149000000,25210438,...,1.051,0.975,1.0842,1.065,1.1020,1.032,1.217,1.241,1.154,1.341
20220502,903,20900.0,20950.0,20900.0,20950.0,294.0,-200,25210000,528149000000,25210438,...,1.051,0.975,1.0842,1.065,1.1020,1.032,1.217,1.241,1.154,1.341
20220502,904,20900.0,20950.0,20900.0,20950.0,64.0,-200,25210000,528149000000,25210438,...,1.051,0.975,1.0842,1.065,1.1020,1.032,1.217,1.241,1.154,1.341
20220502,905,20950.0,20950.0,20950.0,20950.0,8.0,-200,25210000,528149000000,25210438,...,1.051,0.975,1.0842,1.065,1.1020,1.032,1.217,1.241,1.154,1.341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221121,1517,29050.0,29050.0,29000.0,29000.0,10464.0,1800,109142000,3181489000000,109142293,...,2.297,2.193,2.2710,2.272,2.3255,2.224,2.204,2.271,2.167,2.253
20221121,1518,29000.0,29050.0,29000.0,29000.0,24110.0,1800,109142000,3181489000000,109142293,...,2.297,2.193,2.2710,2.272,2.3255,2.224,2.204,2.271,2.167,2.253
20221121,1519,29000.0,29050.0,29000.0,29050.0,19074.0,1800,109142000,3181489000000,109142293,...,2.297,2.193,2.2710,2.272,2.3255,2.224,2.204,2.271,2.167,2.253


In [26]:
X_stock_df.value_counts().isnull().sum()

0

In [25]:
len(X_stock_df.columns)

187